In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [3]:
# !pip install pyarrow

In [4]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
import sklearn

In [7]:
#Q1
len(df.columns)

19

In [8]:
#Q2
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

duration_std_dev = df['duration'].std()
print(duration_std_dev)

42.594351241920904


In [9]:
#Q3
df2 = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
len(df2)/len(df)

0.9812202822125979

In [10]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
#Q4
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [12]:
X_train.shape

(3066766, 518)

In [13]:
#5
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f'Train RMSE: {mean_squared_error(y_train, y_pred, squared=False)}')

Train RMSE: 41.99649230697585


In [14]:
dftest = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

dftest['tpep_pickup_datetime'] = pd.to_datetime(dftest['tpep_pickup_datetime'])
dftest['tpep_dropoff_datetime'] = pd.to_datetime(dftest['tpep_dropoff_datetime'])
dftest['duration'] = (dftest['tpep_dropoff_datetime'] - dftest['tpep_pickup_datetime']).dt.total_seconds() / 60
dftest = dftest[(dftest['duration'] >= 1) & (dftest['duration'] <= 60)]

dftest[categorical] = dftest[categorical].astype(str)

test_dicts = dftest[categorical].to_dict(orient='records')

X_test = dv.transform(test_dicts)

In [16]:
y_pred_test = lr.predict(X_test)

y_test = dftest[target].values

print(f'Test RMSE: {mean_squared_error(y_test, y_pred_test, squared=False)}')

Test RMSE: 7.945262521764321
